<a href="https://colab.research.google.com/github/varshil009/FaceInpainting/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


## THE GOAL IS MAKE A CLASSIFICATION MODEL THAT CAN IDENTIFY FACES WHEN THEY ARE OCCLUDED, BUT BEFORE THAT THE MODEL SHOULD HAVE INFORMATION ABOUT FACES

In [39]:
import torch
import torch.nn as nn
import torch.functional as F
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision import models

from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt


### model 1

In [3]:
class classfier(nn.Module):
    def __init__(self):
        super().__init__()

        self.convolution = nn.Sequential(
            # image shape = 218, 178, 3
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1, dilation=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(8),  # Replacing LayerNorm

            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1, dilation=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(16),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, dilation=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, dilation=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, dilation=2),
            nn.AdaptiveAvgPool2d((4, 3)),  # Fixed output size
            nn.BatchNorm2d(128)  # Normalizing before fully connected layers
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 3, 512),  # Correct input size: 1536
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 10)  # Assuming 10 classes
        )

    def forward(self, x):
        x = self.convolution(x)
        x = self.classifier(x)
        return x

### model 2

In [32]:
class FaceRecognizer(nn.Module):
    def __init__(self):
        super(FaceRecognizer, self).__init__()
        self.convolution = models.efficientnet_b0(pretrained=True)
        self.convolution.fc = nn.Identity() # removes fully connected dense layer
        self.classifier = nn.Sequential(

            nn.Linear(1000, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.convolution(x)
        x = self.classifier(x)
        return x

## make a dataloder that creates batches for data processing

In [5]:
def create_data_loaders_not_used(
    data_dir,
    batch_size=10,
    image_size=(218, 178),
    test_split=0.2
):
    """
    Create data loaders for numeric class folders

    Expected folder structure:
    data_dir/
    ├── 0/
    │   ├── anyimagename1.jpg
    │   ├── anyimagename2.jpg
    ├── 1/
    │   ├── anyimagename1.jpg
    │   ├── anyimagename2.jpg
    """

    # Transformations
    data_transforms = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Load entire dataset
    full_dataset = datasets.ImageFolder(
        root=data_dir,
        transform=data_transforms
    )

    # Get total number of samples
    dataset_size = len(full_dataset)

    # Calculate validation size
    val_size = int(dataset_size * test_split)
    train_size = dataset_size - val_size

    # Split dataset
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset,
        [train_size, val_size]
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )

    # Get class names (numeric in this case)
    class_names = [int(s) for s in ['2820', '3227', '3699', '3745', '3782', '4887', '6568', '8968', '9152', '9256']]

    return train_loader, val_loader, class_names

def create_data_loaders(
    data_dir,
    batch_size=10,
    image_size=(218, 178),
    test_split=0.2
):
    """
    Create data loaders for numeric class folders.

    Expected folder structure:
    data_dir/
    ├── 2820/
    │   ├── anyimagename1.jpg
    │   ├── anyimagename2.jpg
    ├── 3227/
    │   ├── anyimagename1.jpg
    │   ├── anyimagename2.jpg
    """

    # Transformations
    data_transforms = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Load entire dataset
    full_dataset = datasets.ImageFolder(
        root=data_dir,
        transform=data_transforms
    )

    # Extract class mappings
    class_to_idx = full_dataset.class_to_idx  # Dict {class_name: index}
    idx_to_class = {v: int(k) for k, v in class_to_idx.items()}  # Reverse mapping {index: class_name}

    # Get total number of samples
    dataset_size = len(full_dataset)

    # Calculate validation size
    val_size = int(dataset_size * test_split)
    train_size = dataset_size - val_size

    # Split dataset
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset,
        [train_size, val_size]
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=1
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )

    return train_loader, val_loader, idx_to_class  # ✅ Now returning idx_to_class for predictions

######################################################################################################################################################################
##########################################################################################################################################################################
##########################################################################################################################################################################

# Optional: Data augmentation
def get_augmented_transforms(image_size=(218, 178)):
    """
    More comprehensive data augmentation
    """
    return transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

# Additional utility to count samples per class
def count_samples_per_class(data_dir):
    """
    Count number of samples in each class folder
    """
    class_counts = {}
    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        if os.path.isdir(class_path):
            num_samples = len(os.listdir(class_path))
            class_counts[class_folder] = num_samples

    return class_counts

# Print class distribution
def print_class_distribution(data_dir):
    """
    Print number of samples in each class
    """
    class_counts = count_samples_per_class(data_dir)
    print("Class Distribution:")
    for cls, count in class_counts.items():
        print(f"Class {cls}: {count} samples")

In [6]:
data_dir = "/content/drive/MyDrive/CelebA_subset"

# Create data loaders
train_loader, val_loader, class_names = create_data_loaders(data_dir)

# Inspect first batch
for images, labels in train_loader:
    print("Batch images shape:", images.shape)
    print("Batch labels:", labels)
    print("Class names:", class_names)
    break

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Batch images shape: torch.Size([10, 3, 218, 178])
Batch labels: tensor([1, 3, 6, 3, 8, 6, 3, 4, 8, 4])
Class names: {0: 2820, 1: 3227, 2: 3699, 3: 3745, 4: 3782, 5: 4887, 6: 6568, 7: 8968, 8: 9152, 9: 9256}


In [ ]:
len(train_loader)

9

## training time !!
![Image](https://media.tenor.com/oGNAlTqpMvYAAAAM/lets-do-this.gif)


In [76]:
"""# entities
device = "cuda" if torch.cuda.is_available() else "cpu"

model1 = classfier()
model1 = model1.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr = 0.001)"""

In [77]:
"""num_epochs = 40

for e in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, label in train_loader:
        images, labels = images.to(device), label.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += label.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total
    print(f"Epoch [{e+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    # now evaluate on validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.2f}%\n")"""

Epoch [1/40], Loss: 2.1085, Train Acc: 0.25%


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Validation Accuracy: 19.70%

Epoch [2/40], Loss: 1.8003, Train Acc: 0.35%
Validation Accuracy: 36.36%

Epoch [3/40], Loss: 1.6385, Train Acc: 0.41%
Validation Accuracy: 48.48%

Epoch [4/40], Loss: 1.3287, Train Acc: 0.51%
Validation Accuracy: 48.48%

Epoch [5/40], Loss: 1.1550, Train Acc: 0.57%
Validation Accuracy: 69.70%

Epoch [6/40], Loss: 1.0539, Train Acc: 0.60%
Validation Accuracy: 62.12%

Epoch [7/40], Loss: 0.9733, Train Acc: 0.68%
Validation Accuracy: 50.00%

Epoch [8/40], Loss: 0.8377, Train Acc: 0.71%
Validation Accuracy: 53.03%

Epoch [9/40], Loss: 0.6395, Train Acc: 0.76%
Validation Accuracy: 66.67%

Epoch [10/40], Loss: 0.5709, Train Acc: 0.81%
Validation Accuracy: 71.21%

Epoch [11/40], Loss: 0.4039, Train Acc: 0.88%
Validation Accuracy: 66.67%

Epoch [12/40], Loss: 0.3088, Train Acc: 0.89%
Validation Accuracy: 63.64%

Epoch [13/40], Loss: 0.2324, Train Acc: 0.91%
Validation Accuracy: 72.73%

Epoch [14/40], Loss: 0.3141, Train Acc: 0.90%
Validation Accuracy: 75.76%

Epoc

In [33]:
# model2
model = FaceRecognizer()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

num_epochs = 40
best_val_acc = 0.0
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for e in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = (correct / total) * 100
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{e+1}/{num_epochs}], Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%")

    # now evaluate on validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # update the best validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

    print(f"Best Validation Accuracy: {best_val_acc:.2f}%\n")

    # learning rate scheduler
    scheduler.step(val_acc)

Epoch [1/40], Loss: 1.6660, Train Acc: 48.30%
Validation Accuracy: 89.39%
Best Validation Accuracy: 89.39%

Epoch [2/40], Loss: 0.6994, Train Acc: 82.26%
Validation Accuracy: 90.91%
Best Validation Accuracy: 90.91%

Epoch [3/40], Loss: 0.4046, Train Acc: 90.57%
Validation Accuracy: 92.42%
Best Validation Accuracy: 92.42%

Epoch [4/40], Loss: 0.3010, Train Acc: 93.96%
Validation Accuracy: 93.94%
Best Validation Accuracy: 93.94%

Epoch [5/40], Loss: 0.2438, Train Acc: 95.47%
Validation Accuracy: 93.94%
Best Validation Accuracy: 93.94%

Epoch [6/40], Loss: 0.2371, Train Acc: 94.34%
Validation Accuracy: 89.39%
Best Validation Accuracy: 93.94%

Epoch [7/40], Loss: 0.1865, Train Acc: 96.23%
Validation Accuracy: 86.36%
Best Validation Accuracy: 93.94%

Epoch [8/40], Loss: 0.2091, Train Acc: 95.09%
Validation Accuracy: 95.45%
Best Validation Accuracy: 95.45%

Epoch [9/40], Loss: 0.1775, Train Acc: 96.98%
Validation Accuracy: 87.88%
Best Validation Accuracy: 95.45%

Epoch [10/40], Loss: 0.1856,

### try that model on occluded face dataset
### make a dataloader for that with labels

In [35]:
data_dir = "/content/drive/MyDrive/occluded_dataset_BIG"
dirs = os.listdir(data_dir)
dirs

['8968',
 '6568',
 '2820',
 '3745',
 '3227',
 '3782',
 '3699',
 '9256',
 '4887',
 '9152']

In [41]:

# Load the same transformation as used in training
data_transforms = transforms.Compose([
    transforms.Resize((218, 178)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the dataset class (without labels for inference)
class FaceDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.true_labels = []

        self.class_names = sorted(os.listdir(root_dir))  # Get class names (numeric)
        self.class_to_idx = {cls_name: int(cls_name) for cls_name in self.class_names}  # Keep numeric class names

        for class_folder in self.class_names:
            class_path = os.path.join(root_dir, class_folder)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_file)
                    self.image_paths.append(img_path)
                    self.true_labels.append(self.class_to_idx[class_folder])  # Store class label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        true_label = self.true_labels[idx]
        image = Image.open(img_path).convert("RGB")  # Ensure it's RGB

        if self.transform:
            image = self.transform(image)

        return image, true_label, img_path  # Return image, actual class (folder name), and path

# Set dataset path and create DataLoader
eval_dir = "/content/drive/MyDrive/occluded_dataset_BIG"  # Update this path
dataset = FaceDataset(root_dir=eval_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

In [51]:
class_names_dic = {0: 2820, 1: 3227, 2: 3699, 3: 3745, 4: 3782, 5: 4887, 6: 6568, 7: 8968, 8: 9152, 9: 9256}

In [57]:
# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Evaluation
correct, total = 0, 0
class_names = dataset.class_names  # Numeric class names

with torch.no_grad():
    for img, true_label, img_path in dataloader:
        img = img.to(device)
        true_label = true_label.item()

        # Get model prediction
        output = model(img)
        predicted_idx = output.argmax(dim=1).item()

        # Check if prediction is correct
        total += 1
        predicted = class_names_dic[predicted_idx]

        if predicted == true_label:
            correct += 1
        #print(predicted_idx, true_label)
        #print(f"Image: {os.path.basename(img_path[0])} → Predicted: {class_names_dic[predicted_idx]}, Actual: {true_label}")

# Calculate Accuracy
accuracy = (correct / total) * 100
print(f"\nModel Accuracy: {accuracy:.2f}%")


Model Accuracy: 99.11%


In [58]:
torch.save(model.state_dict(), "/content/drive/MyDrive/occluded_dataset_BIG/classifier_model.pth")

In [ ]:
# for reuse of model follow this commands
"""
model = Classifier()  # Recreate the model structure
model.load_state_dict(torch.load("model.pth"))
model.eval()  # Set to evaluation mode
"""